# Analysis of seismically active area
**Author:** Jan Michalek (ORCID: https://orcid.org/0000-0002-8057-7541)

**Date:** 2025-06-24

**Occasion:** EPOS Summer School, part of the training materials

**Summary:**
This use case is exploring several datasets from seismology, geodesy and geology analyzing seismogenic faults, moment tensors, geological map and their relation to geodetic (GNSS) observations. Data is provided through webservices and requested online when the code is executed (can not be executed offline). 

Note: This notebook was built with assistance of Gemini (Google, 2025). 

Executability: Please move this notebook from '/data/latest/' folder to root folder '/' before running. 

**References:** 
Google. (2025). Gemini (Advanced, April 2025 version) [Large language model]. https://gemini.google.com/

## List of services
| Service name | Reference (please check [EPOS Platform](https://www.ics-c.epos-eu.org/) for updates) |
| ------------ | --------- |
| European Fault Source Model 2020 - Crustal Faults (OGC WFS) | European Fault Source Model 2020 - Crustal Faults (OGC WFS), provided by INGV - Istituto Nazionale di Geofisica e Vulcanologia - IT, https://creativecommons.org/licenses/by/4.0/, 10.13127/efsm20. Accessed on 23-06-2025 through the EPOS Data Portal (https://www.epos-eu.org/dataportal) | 
| Geological Feature View Service (EGDI Geological Map 1:1,000,000) | Geological Feature View Service (EGDI Geological Map 1:1,000,000), provided by BGR - Bundesanstalt für Geowissenschaften und Rohstoffe - DE, BGS - British Geological Survey, UK Research and Innovation - GB, BRGM - Bureau de Recherches Géologiques et Minières - FR, CGS - Czech Geological Survey - CZ, CSD - Continental Shelf Department - MT, GBA - Geological Survey of Austria - AT, GEUS - De Nationale Geologiske Undersøgelser for Danmark og Grønland - DK, GSI - Geological Survey of Ireland - IE, GTK - Geological Survey of Finland - FI, GeoZS - Geoloski Zavod Slovenije - SI, Geological Survey of Belgium - BE, H.S.G.M.E. - Hellenic Survey of Geology and Mineral Exploration - GR, IGME - Instituto Geológico y Minero de España - ES, ISPRA - Istituto Superiore per la Protezione e la Ricerca Ambientale - Portale del Servizio Geologico d'Italia - IT, LNEG - Laboratório Nacional de Energia e Geologia - PT, Mining and Geological Survey of Hungary - HU, NGU - Geological Survey of Norway - NO, NI - Icelandic Institute of Natural History - IS, PGI - Polish Geological Institute – National Research Institute - PL, SGL - Service Géologique du Luxembourg - LU, SGU - Sveriges Geologiska Undersökning - SE, SGUDS - State Geological Institute of Dionyz Stur - SK, TNO - Geological Survey of the Netherlands - NL, https://creativecommons.org/licenses/by/4.0/. Accessed on 23-06-2025 through the EPOS Data Portal (https://www.epos-eu.org/dataportal) | 
| Moment tensor data for modern earthquakes (2013-present) | Moment tensor data for modern earthquakes (2013-present), provided by EMSC - European-Mediterranean Seismological Centre - FR, https://creativecommons.org/licenses/by/4.0/. Accessed on 23-06-2025 through the EPOS Data Portal (https://www.epos-eu.org/dataportal) |
| GNSS Stations with Products | GNSS Stations with Products, provided by UBI - University of Beira Interior - PT, https://creativecommons.org/licenses/by/4.0/. Accessed on 23-06-2025 through the EPOS Data Portal (https://www.epos-eu.org/dataportal) |
| EPOS GNSS Daily Position Time Series from UGA-CNRS | EPOS GNSS Daily Position Time Series from UGA-CNRS, provided by UGA - Université Grenoble Alpes - FR, https://creativecommons.org/licenses/by/4.0/, 10.17178/GNSS.products.EPOS.2019. Accessed on 23-06-2025 through the EPOS Data Portal (https://www.epos-eu.org/dataportal) |



In [ ]:
# Install libraries - uncomment the following line if running first time or having errors about missing libraries 
!pip install geopandas fiona shapely mapclassify folium owslib xmltodict obspy


In [ ]:
# Import libraries
import os, sys
import warnings
import re
import math
import pprint
import requests
import warnings
import json
import matplotlib.pyplot as plt
from datetime import datetime
import xml.etree.ElementTree as ET
import geopandas as gpd
import folium # interactive plotting
import numpy 

# Define some abbreviations
pp = pprint.PrettyPrinter(indent=4)

# Turn off warnings (suppresses output from certification verification when verify=false)
warnings.simplefilter("ignore")

In [ ]:
# INPUT - define your input parameters
# This is the only cell requiring your input

# Geolocation - Santorini area, Greece
lat_min = 36.06571
lat_max = 37.06319
lon_min = 24.88953
lon_max = 26.38367
bbox = (lon_min, lat_min, lon_max, lat_max) # tuple type

# Define time range - NOT USED NOW
#end_time = datetime.datetime.now()
#start_time = end_time - datetime.timedelta(days=90)

## Show geological map - static image
There are several options to visualize map in notebooks. The example below is creating statis image but later on we will visualize the same content using interactice map view.

In [ ]:
from owslib.wfs import WebFeatureService
from owslib.wms import WebMapService
from owslib.util import Authentication
from IPython.display import Image

service_url_GeologicUnit = 'https://data.geoscience.earth/api/wmsGeologicUnit?'
wms = WebMapService(service_url_GeologicUnit, version='1.3.0', auth=Authentication(verify=False))
#list(wms.contents)
payload = {
    'layers' : ['GeologicUnitView_Lithology'],
    'format' : 'image/png',
    'srs'    : 'EPSG:4326',
    'bbox'   : (bbox), 
    'size'   : (800,500) }

img = wms.getmap(**payload)
Image(img.read())

## Interactive plotting
Folium library is used to create interactive map directly in the notebook. Further customization of the map view is possible, explore 'folium' documentation for details.

In [ ]:
# Determine the center of the map based on the average latitude and longitude
avg_lat = ( lat_max + lat_min ) / 2
avg_lon = ( lon_max + lon_min ) / 2

# Create a Folium map centered around the average coordinates
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=9)

## Geological map

In [ ]:
# Add a WMS tile layer - GEOLOGY
wms_layer = folium.raster_layers.WmsTileLayer(
    url='https://data.geoscience.earth/api/wmsGeologicUnit?',  # Replace with the actual WMS service URL
    layers='GeologicUnitView_Lithology',  # Replace with the name of the layer you want to display
    #styles='YOUR_STYLE',  # Optional: specify a style for the layer
    fmt='image/png',  # Or 'image/jpeg', etc., depending on the WMS service
    transparent=True,  # Set to True if you want the background to be transparent
    version='1.3.0',  # Or the version supported by the WMS service
    name='Geological lithology',  # Name to display in the layer control
    overlay=True,  # Add as an overlay (True) or base layer (False)
    control=True,  # Add to the layer control
    show=True,  # Initially show the layer
    attr='EGDI'  # Optional: Add attribution for the data source
).add_to(m)


## Crustal faults

In [ ]:
# Add WFS layer for Crustal Faults
wfs_url = 'https://services.seismofaults.eu/EFSM20/ows'
layer_name = 'efsm20_cf_top'  # Adjust as needed
bbox_2 = (lat_min, lon_min, lat_max, lon_max)
bbox_str = ','.join(str(val) for val in bbox_2)

# Define a style function that returns a fixed color
def style_all(feature):
    return {'color': 'red', 'weight': 2, 'fillOpacity': 0.7}
    
try:
    # Try reading directly from the WFS URL (may not work for all services)
    gdf = gpd.read_file(f"{wfs_url}?service=WFS&version=2.0.0&request=GetFeature&typeName={layer_name}&outputFormat=GML2&srsName=EPSG:4326&bbox={bbox_str}")

    if not gdf.empty:  
        geojson_data = gdf.to_json()  # Convert the GeoDataFrame to GeoJSON     
        folium.GeoJson(geojson_data, name='Crustal Faults WFS', style_function=style_all).add_to(m)  # Add the GeoJSON data to the existing Folium map   
        folium.LayerControl().add_to(m) # Add layer control to toggle the GeoJSON layer           
    
    else:
        print("No features read from WFS.")

except Exception as e:
    print(f"Error reading WFS data with GeoPandas: {e}")
display(m)

## Focal mechanisms of earthquakes
Request moment tensor data from Seismic Portal through a webservice. Beach balls are are created and saved as images and then used as map markers, catagorized by colors according to CLVD values. Higher CLVD values can serve as indicator for magma intrusion.

In [ ]:
# Load data
# https://www.seismicportal.eu/mtws/api/search?minlat=36.06571&maxlat=37.06319&minlon=24.88953&maxlon=26.38367&preferredOnly=true&format=json&limit=1
#import pygmt
from obspy.imaging.beachball import beachball

num_eq = 5    # MODIFY for more MT (limit 1000; note: recommended up to 50; the more the slower the response in interactive plot) 
baseURL = 'https://www.seismicportal.eu/mtws/api/search?'
param_str = 'minlat=' + str(lat_min) + '&maxlat=' + str(lat_max) + '&minlon=' + str(lon_min) + '&maxlon=' + str(lon_max)
param_str = param_str + '&preferredOnly=true&format=json&limit=' + str(num_eq)
content = requests.get(baseURL + param_str)
print(content.url)
EQ = json.loads(content.text)

!mkdir bb
for event in EQ:
    # Generate beach ball image using obspy 
    mt = [event['mt_strike_1'],event['mt_dip_1'],event['mt_rake_1']]
    #print(mt)
    outfile = './bb/' + event['ev_unid'] + '.png'
    beachball(mt, size=5, linewidth=2, facecolor='r', outfile=outfile)
    
    # Create an HTML string for the popup
    html = f'''
        <b>Time:</b> {event['ev_event_time']}<br>
        <b>Magnitude:</b> {event['ev_mag_value']}<br>
        <b>Depth:</b> {event['ev_depth']}<br>
    '''
    # Custom icon
    icon_image = folium.CustomIcon(
        icon_image= outfile,  # Replace with the actual path to your image file
        icon_size=(30, 30),  # Optional: specify the size of the icon (width, height) in pixels
        icon_anchor=(15, 30),  # Optional: the point on the icon that aligns with the lat/lon
        popup_anchor=(0, -30)   # Optional: where the popup opens relative to the icon_anchor
    )
    # Create a popup with the HTML content
    popup = folium.Popup(html, max_width=150)

    # Add a marker with the popup
    folium.Marker(
        location=[event['ev_latitude'], event['ev_longitude']],
        popup=popup,
        tooltip=event['ev_unid'],
        icon=icon_image
    ).add_to(m)
display(m)

## GNSS stations
List of existing GNSS stations is requested through webservice and stations are added to the map. Each station already contains link(s) to daily position timeseries which are used later.

In [ ]:
baseURL = 'https://gnssproducts.epos.ubi.pt/GlassFramework/webresources/stations/v2/station/'
bbox_str = 'bbox/' + '/'.join(str(val) for val in bbox)
add_param = '?with=2'
content = requests.get(baseURL + bbox_str + add_param)
#print(content.url)
GNSS = json.loads(content.text)
#print('GNSS dictionary:')
#pp.pprint(GNSS)     # check content

stations = []
for feature in GNSS['features']:
    station_id = feature['properties'].get('GNSS Station ID')
    title = feature['properties'].get('Title')
    city = feature['properties'].get('City')
    latitude = feature['properties'].get('Latitude')
    longitude = feature['properties'].get('Longitude')
    data_links = feature['properties'].get('Links TimeSeries')
    networks = feature['properties'].get('Networks')
    data_providers = feature['properties'].get('TimeSeries Data Providers')
    
    station_info = {
        'id': station_id,
        'title': title,
        'city': city,
        'latitude': latitude,
        'longitude': longitude,
        'data_links': data_links,
        'networks' : networks,
        'data_providers' : data_providers
    }
    stations.append(station_info)

#print(stations)
#stations

# Plot GNSS stations in map
# Add markers for each station
for station in stations:
    folium.Marker(
        [station['latitude'], station['longitude']],
        popup=f"<b>{station['title']}</b><br>City: {station['city']}<br>Networks: {station['networks']}<br>Data Providers: {station['data_providers']}",
        tooltip=station['id']
    ).add_to(m)
    
display(m)

# Or save the map to an HTML file
# m.save('map.html')

 # Explore daily position timeseries from GNSS stations
 

In [ ]:
# Plot GNSS timeseries
for item in stations:
    if 'data_links' in item:
        for link_name, link_url in item['data_links'].items():
            link_url_enz = link_url.replace("/xyz/", "/enu/") # requesting East, North, Up data instead of XYZ; units in mm
            content = requests.get(link_url_enz)
            payload = json.loads(content.text)
            
            time_values = [datetime.fromisoformat(t.replace('Z', '+00:00')) for t in payload['domain']['axes']['t']['values']]
            x_values = payload['ranges']['E']['values']
            y_values = payload['ranges']['N']['values']
            z_values = payload['ranges']['U']['values']
            
            plt.figure(figsize=(10, 4))
            plt.plot(time_values, x_values, label='E', marker='.', markersize=2, linewidth=0.7)
            plt.plot(time_values, y_values, label='N', marker='.', markersize=2, linewidth=0.7)
            plt.plot(time_values, z_values, label='U', marker='.', markersize=2, linewidth=0.7)
            plt.xlabel('Time')
            plt.ylabel('Value (mm)')
            plt.title('GNSS Time Series for ' + item['title'] + ' provided by: ' + link_name)
            plt.grid(True)
            plt.legend()
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()

## Analyze moment tensors in time
Note: Please adjust **start_date** and **end_date** to plot range you are interested in.

In [ ]:
import pandas as pd

# Convert the dictionary to a DataFrame
df = pd.DataFrame(EQ)

# Print the DataFrame
#print(df)

# Set the zoom window for February 2025
start_date = pd.to_datetime('2025-01-31')
end_date = pd.to_datetime('2025-06-28')

# Convert 'mt_centroid_time' to datetime objects
# Handle potential errors due to the incomplete timestamp in the last entry
df['time'] = pd.to_datetime(df['ev_event_time'], errors='coerce')

# Sort the DataFrame by time
df = df.sort_values(by='time')
df = df.dropna(subset=['time']) # Remove rows with invalid time

# Calculate the actual seismic moment using mt_m0 and mt_m0_exp
df['seismic_moment'] = df['mt_m0'] * (10 ** df['mt_m0_exp'])

# Create the cumulative seismic moment vector
df['cumulative_seismic_moment'] = df['seismic_moment'].cumsum()

# Create the figure with subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Plot event magnitude on the first subplot
axs[0].plot(df['time'], df['cumulative_seismic_moment'], marker='', linestyle='-', label='Cumulative Seismic Moment', color='blue')
axs[0].set_ylabel('Seismic Moment [Nm]')
axs[0].set_title('Cumulative Seismic Moment')
axs[0].grid(True)
axs[0].legend()

# Plot DC, CLVD, and ISO on the second subplot
axs[1].plot(df['time'], df['mt_per_dc'], marker='x', linestyle='', label='Double Couple (%)', color='green')
axs[1].plot(df['time'], df['mt_per_clvd'], marker='s', linestyle='--', label='CLVD (%)', color='red')
axs[1].plot(df['time'], df['mt_per_iso'], marker='d', linestyle='', label='Isotropic (%)', color='purple')
axs[1].set_ylabel('Percentage (%)')
axs[1].set_xlabel('Time (UTC)')
axs[1].grid(True)
axs[1].legend()
axs[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.xlim(start_date, end_date)
plt.show()